In [1]:
import numpy as np
import pandas as pd 

In [2]:
train = pd.read_csv('D:/Data Science/Course_Quera/Machine Learning/Class II/Project2/gender_detection/data/train_data.csv')
test = pd.read_csv('D:/Data Science/Course_Quera/Machine Learning/Class II/Project2/gender_detection/data/test_data.csv')

In [3]:
train.head()

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
0,man,2,Farshid,mr_gh_farshid,دردا ک در این بادیه بسیار دویدیم...\nGlory man...,1604.0,1407.0,0.0,0.0,0.0
1,woman,2,zahr@72,zahra.roozbahani72,"خواهی که زکوچ در امان برگردی\nباید که به جان ,...",67.0,501.0,0.0,0.0,0.0
2,woman,2,ms farahnaz♥,___lady.farahnazi.__,"Having you, is all I wish for \nداشتنت، تمامِ...",0.0,0.0,0.0,0.0,0.0
3,woman,1,Lena.mommy farzan,mommy.lena3361,دردونه من لنا کوچولو,0.0,0.0,0.0,0.0,0.0
4,woman,2,Narsis Asadollahi,_l.aurora.l_,I am an animation student\n🎧🎼🎨⚓️🤍 \n@general.m...,200.0,328.0,0.0,0.0,0.0


In [4]:
test.head()

,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
0,2,Mehrnoosh Rahimi,mehrnoosh_hooka,🕉♋️\n 🎂 Mehr🎂\nگرچه بنی ادم اعضای یک پیکر اند\...,791.0,174.0,0.0,0.0,0.0
1,4,Mehdi,mehdil_eng,"Civil engineer(Shiraz Uni) , Book , Travel...",0.0,0.0,0.0,0.0,0.0
2,2,MohammadHossein ahmadizadeh,madh0sin,(به نام‌خدایی‌که داده اش‌نعمت‌است‌ونداده اش حک...,1902.0,3436.0,1.0,0.0,0.0
3,1,Sara Noori,saray_nri,▪️UI,114.0,173.0,0.0,0.0,0.0
4,2,Shi Ma,shima_syd,Lives in turkey\nThe married 👫💑👪,0.0,0.0,0.0,0.0,0.0


In [5]:
import re

# convert Column Values to Lowercase:
# df['column_name'] = df['column_name'].str.lower()
# df['column_name'] = df['column_name'].apply(str.lower)
# df['column_name'] = df['column_name'].map(str.lower)
# df['column_name'] = df['column_name'].str.casefold()

# remove numbers from string terms: Replace all occurring digits in the strings with nothing 
# df['column_name'] = df['column_name'].str.replace('\d+', '')  #\d stands for "any digit" and + stands for "one or more"
# df.replace('\d+', '', regex=True, inplace=True)

# delete symbols
# df['$name$'] = df['$name$'].str.replace(r'\W', '', regex=True)
# df['$name$'] = df['$name$'].str.replace(r'[^a-z0-9]','', regex=True, flags=re.IGNORECASE) #remove also the underscores

#re.sub() function replaces one or many matches with a string in the given text: re.sub(pattern, replace, string, count=0, flags=0)

def clean_text(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'&amp;', 'and', text)  
    text = re.sub(r'@', 'a', text)
    text = re.sub(r'[^a-zA-Z\s.]', '', text) # \s: contains a white space character. a-zA-Z: any character alphabetically between a and z, lower case OR upper case
    return text

columns = ['fullname', 'username']
for col in columns:
    train[col] = train[col].astype(str).apply(clean_text)
    test[col] = test[col].astype(str).apply(clean_text)


train['is_business'].fillna(train['is_business'].mode()[0], inplace=True)

train['fullname'].fillna(train['username'], inplace=True)
test['fullname'].fillna(test['username'], inplace=True)

def count_words(text):
    return len(text.split())

train['bio_word_count'] = train['biography'].astype(str).apply(count_words)
test['bio_word_count'] = test['biography'].astype(str).apply(count_words)

In [6]:
pip install emoji


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# add new feature to show number of emoji
import emoji
from emoji import emoji_count

def count_emojis(text):
    return emoji_count(text)

train['emoji_count'] = train['biography'].apply(count_emojis)
test['emoji_count'] = test['biography'].apply(count_emojis)

In [8]:
# convert text columns to vector
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

# the union of these three columns: train.fullname + ' ' + train.username + ' ' + train.biography
tfidf.fit(train.fullname + ' ' + train.username + ' ' + train.biography)
tfidf.transform(test.fullname + ' ' + test.username + ' ' + test.biography)

<2000x32558 sparse matrix of type '<class 'numpy.float64'>'
	with 17451 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
train['gender'] = le.fit_transform(train['gender'])

y = train.gender
X = train.drop(['gender'], axis=1)

# modeling to add new feature based on text columns
from xgboost import XGBClassifier
# create model 
first_model = XGBClassifier()
# fit model
first_model.fit(tfidf.transform(X.fullname + ' ' + X.username + ' ' + X.biography), y)
# make predictions to extract new feature
train['gender_by_txt'] = first_model.predict(tfidf.transform(train.fullname + ' ' + train.username + ' ' + train.biography))
test['gender_by_txt'] = first_model.predict(tfidf.transform(test.fullname + ' ' + test.username + ' ' + test.biography))

In [10]:
train.drop(['fullname', 'username', 'biography'], axis=1, inplace=True)
train.head()

,gender,age,follower_count,following_count,is_business,is_verified,is_private,bio_word_count,emoji_count,gender_by_txt
0,0,2,1604.0,1407.0,0.0,0.0,0.0,14,3,0
1,1,2,67.0,501.0,0.0,0.0,0.0,14,0,1
2,1,2,0.0,0.0,0.0,0.0,0.0,17,8,1
3,1,1,0.0,0.0,0.0,0.0,0.0,4,0,1
4,1,2,200.0,328.0,0.0,0.0,0.0,7,5,1


In [11]:
test.drop(['fullname', 'username', 'biography'], axis=1, inplace=True)
test.head()

,age,follower_count,following_count,is_business,is_verified,is_private,bio_word_count,emoji_count,gender_by_txt
0,2,791.0,174.0,0.0,0.0,0.0,16,9,0
1,4,0.0,0.0,0.0,0.0,0.0,10,0,0
2,2,1902.0,3436.0,1.0,0.0,0.0,12,7,1
3,1,114.0,173.0,0.0,0.0,0.0,1,1,1
4,2,0.0,0.0,0.0,0.0,0.0,6,3,1


In [12]:
from sklearn.model_selection import train_test_split
y = train.gender
X = train.drop(['gender'], axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X,y , test_size=0.2)

final_model = XGBClassifier()
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_valid)

# evaluate model
from sklearn.metrics import  f1_score, classification_report, confusion_matrix
f1_score(y_valid, y_pred, average='weighted')

0.8336413974360778

In [13]:
# predict test samples
prediction = final_model.predict(test)
submission = pd.DataFrame(prediction, columns=['gender']).replace({1: 'woman', 0: 'man'})  
submission

,gender
0,man
1,man
2,woman
3,woman
4,woman
...,...
1995,woman
1996,woman
1997,man
1998,man
